### 1. Transactions Completed During Each Marketing Campaign

```sql
SELECT
    M.campaign_name,
    COUNT(T.transaction_id) AS transaction_completed
FROM
    transactions AS T
INNER JOIN
    marketing_campaigns M
ON
    T.Product_id = M.Product_id
    AND T.purchase_date BETWEEN M.start_date AND M.end_date
GROUP BY
    M.campaign_name;
```

### 2. Product with the Highest Sales Quantity

```sql
WITH cte_top AS (
    SELECT TOP 1
        product_id,
        SUM(quantity) AS total
    FROM
        transactions
    GROUP BY
        product_id
    ORDER BY
        total DESC
)
SELECT
    B.product_id,
    B.product_name,
    A.total AS highest_sales_quantity
FROM
    cte_top A
LEFT JOIN
    sustainable_clothing B
ON
    A.product_id = B.product_id;
```

### 3. Total Revenue Generated from Each Marketing Campaign

```sql
SELECT
    A.campaign_name,
    SUM(B.price * C.quantity) AS total_revenue_generated
FROM
    marketing_campaigns A
INNER JOIN
    sustainable_clothing B
ON
    A.product_id = B.product_id
INNER JOIN
    transactions C
ON
    C.product_id = B.product_id
GROUP BY
    A.campaign_name;
```

### 4. Top-Selling Product Category Based on Total Revenue Generated

```sql
SELECT TOP 1
    B.category AS Top_selling_Product_category,
    SUM(B.price * C.quantity) AS total_revenue_generated
FROM
    sustainable_clothing B
INNER JOIN
    transactions C
ON
    C.product_id = B.product_id
GROUP BY
    B.category
ORDER BY
    total_revenue_generated DESC;
```

### 5. Products with Higher Quantity Sold Than Average

```sql
SELECT DISTINCT
    product_id,
    quantity
FROM
    transactions
WHERE
    quantity > (SELECT AVG(quantity) FROM transactions);
```

### 6. Average Revenue Generated per Day During Marketing Campaigns

```sql
WITH cte_a AS (
    SELECT
        T.Product_id,
        M.campaign_name,
        COUNT(T.transaction_id) AS frequency
    FROM
        transactions AS T
    INNER JOIN
        marketing_campaigns M
    ON
        T.Product_id = M.Product_id
        AND T.purchase_date BETWEEN M.start_date AND M.end_date
    GROUP BY
        T.Product_id,
        M.campaign_name
),
cte_b AS (
    SELECT
        C.product_id,
        SUM(A.quantity * B.price) AS total_revenue_generated
    FROM
        transactions A
    INNER JOIN
        sustainable_clothing B
    ON
        A.product_id = B.product_id
    INNER JOIN
        marketing_campaigns C
    ON
        C.product_id = A.product_id
        AND A.purchase_date BETWEEN C.start_date AND C.end_date
    GROUP BY
        C.product_id
)
SELECT
    A.campaign_name,
    (total_revenue_generated / frequency) AS Average_revenue_generated_per_campaign
FROM
    cte_a A
INNER JOIN
    cte_b B
ON
    A.product_id = B.product_id;
```

### 7. Percentage Contribution of Each Product to Total Revenue

```sql
SELECT
    sc.product_id,
    sc.product_name,
    SUM(t.quantity * sc.price) AS revenue,
    (SUM(t.quantity * sc.price) / total_revenue.total) * 100 AS percentage_contribution
FROM
    sustainable_clothing sc
JOIN
    transactions t
ON
    sc.product_id = t.product_id
CROSS JOIN (
    SELECT
        SUM(quantity * price) AS total
    FROM
        sustainable_clothing sc
    JOIN
        transactions t
    ON
        sc.product_id = t.product_id
) AS total_revenue
GROUP BY
    sc.product_id,
    sc.product_name,
    total_revenue.total
ORDER BY
    revenue DESC;
```

### 8. Comparison of Average Quantity Sold During vs. Outside Marketing Campaigns

```sql
WITH campaign_sales AS (
    SELECT
        t.product_id,
        CASE
            WHEN t.product_id IN (SELECT product_id FROM marketing_campaigns) THEN 'During Campaigns'
            ELSE 'Outside Campaigns'
        END AS campaign,
        t.quantity
    FROM
        transactions t
)
SELECT
    campaign,
    AVG(quantity) AS average_quantity_sold
FROM
    campaign_sales
GROUP BY
    campaign
ORDER BY
    campaign;
```

### 9. Comparison of Revenue Generated During vs. Outside Marketing Campaigns

```sql
WITH campaign_revenue AS (
    SELECT
        t.product_id,
        CASE
            WHEN t.product_id IN (SELECT product_id FROM marketing_campaigns) THEN 'During Campaigns'
            ELSE 'Outside Campaigns'
        END AS campaign,
        t.quantity * sc.price AS revenue
    FROM
        transactions t
    JOIN
        sustainable_clothing sc
    ON
        t.product_id = sc.product_id
)
SELECT
    campaign,
    SUM(revenue) AS total_revenue
FROM
    campaign_revenue
GROUP BY
    campaign
ORDER BY
    campaign;
```

### 10. Ranking Products by Average Daily Quantity Sold

```sql
WITH daily_quantity AS (
    SELECT


        product_id,
        SUM(quantity) AS total_quantity,
        COUNT(DISTINCT purchase_date) AS total_days,
        SUM(quantity) / COUNT(DISTINCT purchase_date) AS average_daily_quantity
    FROM
        transactions
    GROUP BY
        product_id
)
SELECT
    product_id,
    average_daily_quantity,
    RANK() OVER (ORDER BY average_daily_quantity DESC) AS product_rank
FROM
    daily_quantity
ORDER BY
    product_rank;
```

## Valuable Insights:

1. "Organic Cotton Sweater" is the product with the highest sales quantity.
2. The "Summer Sale" campaign generated the most revenue, indicating the success of seasonal discounts. The "Super Save" campaign had lower sales, suggesting a need for improved discounts for bulk purchases.
3. The top-selling product category in sustainable clothing is "Bottoms," with the highest revenue of $1289.79.
4. Top revenue-generating products include "Recycled Polyester Jacket," "Bamboo Yoga Leggings," "Organic Cotton Sweater," "Linen Jumpsuit," and "Recycled Denim Jeans."
5. The highest average revenue generated per day was during the "Summer Sale" campaign.
6. "Recycled Denim Jeans" contributed the highest percentage (13.70%) to the total revenue, while "Hemp Crop Top" had the lowest contribution (0.53%).
7. More revenue was generated outside campaigns than during campaigns, possibly due to only three products (Product_id 2, 7, and 10) being included in campaigns.
